In [ ]:
import os
import numpy as np
import cv2
from dlclive import DLCLive, Processor

In [ ]:
model_path = '/model'
video_path = '/data/videos/video.MOV'
frame_path = '/data/frames'

In [ ]:
# 챗봇에서 보내진 비디오 저장
# 이 코드에선 임의로 video_path 이하에 video.MOV로 저장하고 test

In [ ]:
# 비디오 프레임 추출
vcap = cv2.VideoCapture(video_path)
ret, image = vcap.read()

cnt = 0
width = vcap.get(3)
height = vcap.get(4)
print(width, height)

#WIDTH = 267
#HEIGHT = 150
RESIZIED_RATIO = 0.3
fps = vcap.get(cv2.CAP_PROP_FPS)

if os.path.exists(frame_path):
    for frame in os.scandir(frame_path):
        os.remove(frame.path)

while(vcap.isOpened()):
    ret, image = vcap.read()
    if ret:
        #image = cv2.resize(image, dsize=(WIDTH,HEIGHT), interpolation=cv2.INTER_AREA)
        image = cv2.resize(image, dsize=(0,0), fx=RESIZIED_RATIO, fy=RESIZIED_RATIO, interpolation=cv2.INTER_LINEAR)
        #print(vcap.get(1))
        if(int(vcap.get(1)) % (int(fps/5)) == 0):
            cv2.imwrite(frame_path + '/frame%d.png' % cnt, image)
            #cv2.imshow('frame', image)
            #print('/content/gdrive/MyDrive/data/frames/frame%d.png' % cnt +'save succeed')
            cnt += 1
    else:
        print('All the frames are read!')
        break
vcap.release()

In [ ]:
width = int(width * RESIZIED_RATIO)
height = int(height * RESIZIED_RATIO)

In [ ]:
# 추출된 모든 프레임에 대해 keypoint detection

total_points = []
for i in range(cnt):
    video_points = []
    img = cv2.imread(frame_path + '/frame%d.png'%(i))
    #img = cv2.imread(frame)
    dlc_proc = Processor()
    dlc_live = DLCLive(model_path, processor=dlc_proc)
    dlc_live.init_inference(img)
    points = dlc_live.get_pose(img)
    for i in points:
        video_points.append(i[0]/width)
        video_points.append(i[1]/height)
    total_points.append(video_points)

FRAME_LENGTH = 100
NUMBER_OF_POINTS = 30
if len(total_points) >= FRAME_LENGTH:
    total_points = total_points[:FRAME_LENGTH] 
else:
    for i in range(FRAME_LENGTH - len(total_points)):
        total_points.append([0 for i in range(NUMBER_OF_POINTS)])

total_points = np.array(total_points)
#total_points.shape # (100,30)
total_points = np.reshape(total_points, ((1,) + total_points.shape)) # (1,100,30)

In [ ]:
# predict emotion by LSTM model
from tensorflow.keras.models import load_model
trained_lstm_model = load_model(model_path + '/best_model.h5')
prediction = trained_lstm_model.predict(total_points)
predicted_emotion = np.argmax(prediction)

In [ ]:
if predicted_emotion == 0:
    y = '공격성'
elif predicted_emotion == 1:
    y = '공포'
elif predicted_emotion == 2:
    y = '불안/슬픔'
elif predicted_emotion == 3:
    y = '편안/안정'
elif predicted_emotion == 4:
    y = '행복/즐거움'
else:
    y = '화남/불쾌'

print(y) # y: 분석결과 감정